In [9]:
%pip install "tensorflow>=1.7.0"
%pip install tensorflow-hub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

AttributeError: module 'tensorflow_hub' has no attribute 'Module'

In [6]:
embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

AttributeError: module 'tensorflow_hub' has no attribute 'Module'

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

AttributeError: module 'tensorflow_hub' has no attribute 'Module'